In [2]:
pID = '9'



import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import cross_val_score, KFold
import pickle

path = '/Users/lukasgehrke/Documents/publications/2021-fastReach/data/study/eeglab2python/'
data_fname = '/data.csv'
data = pd.read_csv(path+pID+data_fname)

# selected EEG chans
chans_fname = '/chans.csv'
chans = np.array(pd.read_csv(path+pID+chans_fname))
filename = path+pID+'/chans_'+pID+'_eeg.sav'
pickle.dump(chans, open(filename, 'wb'))

# EEG features: 1. windowed means over EEG channels
win_mean_size = 25
win_mean = np.arange(len(data))//win_mean_size
win_mean = data.groupby(win_mean).mean()

df2 = win_mean.pivot(index=['epoch_ix', 'rp_class'], columns='sample', values=win_mean.columns[4:].tolist())
df2.reset_index(inplace=True)
# df2.head()

# EEG
clf = LDA(solver='lsqr', shrinkage='auto')
eeg_dim1 = 2

clf.fit(df2.iloc[:,eeg_dim1:].values, df2.rp_class)
print(clf.coef_.shape)

kfolds = KFold(n_splits=10, random_state=1, shuffle=True) 
cv_results = cross_val_score(clf, df2.iloc[:,eeg_dim1:].values, df2.rp_class, cv=kfolds)
cv_average = cv_results.mean()
print(cv_average)
st_deviation = cv_results.std()
print(st_deviation)

filename = path+pID+'/model_'+pID+'_eeg.sav'
pickle.dump(clf, open(filename, 'wb'))

# Motion
clf = LDA()
mov_data = data.groupby(['epoch_ix']).mean()
clf.fit(mov_data.Motion.values.reshape(-1,1), mov_data.rp_class)

kfolds = KFold(n_splits=10, random_state=1, shuffle=True) 
cv_results = cross_val_score(clf, mov_data.Motion.values.reshape(-1,1), mov_data.rp_class, cv=kfolds)
cv_average = cv_results.mean()
print(cv_average)
st_deviation = cv_results.std()

filename = path+pID+'/model_'+pID+'_motion.sav'
pickle.dump(clf, open(filename, 'wb'))

(1, 300)
0.6544444444444444
0.1690532825640455
1.0
